In [12]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 

In [13]:
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [14]:
data = pd.read_csv('/content/Data_set.csv')

In [15]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Acc_ID                      30000 non-null  int64  
 1   Limit                       30000 non-null  int64  
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   Debt                        29999 non-null  float64
 6   BILL_AMT1                   30000 non-null  int64  
 7   BILL_AMT2                   30000 non-null  int64  
 8   BILL_AMT3                   30000 non-null  int64  
 9   BILL_AMT4                   30000 non-null  int64  
 10  BILL_AMT5                   30000 non-null  int64  
 11  BILL_AMT6                   30000 non-null  int64  
 12  PAY_AMT1                    30000 non-null  int64  
 13  PAY_AMT2                    300

In [16]:
data['default payment next month'].value_counts()


0    23364
1     6636
Name: default payment next month, dtype: int64

In [17]:
default_rate = data['default payment next month'].sum() / data.shape[0]
default_rate *= 100
default_rate


22.12

In [18]:
data_model = data[(data['BILL_AMT6']>0) & (data['Limit']>0) & (data['Debt']>31)]


In [19]:
print("Number of Rows and Columns in Model Development Data: ")



Number of Rows and Columns in Model Development Data: 


In [20]:
default_rate_model = data_model['default payment next month'].sum() / data_model.shape[0]
default_rate_model *= 100
default_rate_model


21.504198612632347

In [21]:
data_model['payment_ratio'] = data_model['PAY_AMT6'] / data_model['BILL_AMT6']


<ipython-input-21-2ec836f7275a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_model['payment_ratio'] = data_model['PAY_AMT6'] / data_model['BILL_AMT6']


In [22]:
payment_ratio_min = data_model['payment_ratio'].min()
payment_ratio_max = data_model['payment_ratio'].max()
payment_ratio_mean = data_model['payment_ratio'].mean()

print("Minimum Payment Ratio: ", payment_ratio_min)
print("Maximum Payment Ratio: ", payment_ratio_max)
print("Mean Payment Ratio: ", payment_ratio_mean)


Minimum Payment Ratio:  0.0
Maximum Payment Ratio:  2643.33
Mean Payment Ratio:  1.7047184027756808


In [23]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Acc_ID                      30000 non-null  int64  
 1   Limit                       30000 non-null  int64  
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   Debt                        29999 non-null  float64
 6   BILL_AMT1                   30000 non-null  int64  
 7   BILL_AMT2                   30000 non-null  int64  
 8   BILL_AMT3                   30000 non-null  int64  
 9   BILL_AMT4                   30000 non-null  int64  
 10  BILL_AMT5                   30000 non-null  int64  
 11  BILL_AMT6                   30000 non-null  int64  
 12  PAY_AMT1                    30000 non-null  int64  
 13  PAY_AMT2                    300

In [24]:
# create rescaled variable
data['LIMIT_BAL1'] = data['Limit'] / 1000

# compute minimum, maximum, and mean of 'Limit' and 'LIMIT_BAL1'
min_limit = data['Limit'].min()
max_limit = data['Limit'].max()
mean_limit = data['Limit'].mean()

min_limit_bal1 = data['LIMIT_BAL1'].min()
max_limit_bal1 = data['LIMIT_BAL1'].max()
mean_limit_bal1 = data['LIMIT_BAL1'].mean()

# print the results
print(f"Minimum value of 'Limit': {min_limit}")
print(f"Maximum value of 'Limit': {max_limit}")
print(f"Mean value of 'Limit': {mean_limit}")
print(f"Minimum value of 'LIMIT_BAL1': {min_limit_bal1}")
print(f"Maximum value of 'LIMIT_BAL1': {max_limit_bal1}")
print(f"Mean value of 'LIMIT_BAL1': {mean_limit_bal1}")

Minimum value of 'Limit': 10100
Maximum value of 'Limit': 1000500
Mean value of 'Limit': 167785.74266666666
Minimum value of 'LIMIT_BAL1': 10.1
Maximum value of 'LIMIT_BAL1': 1000.5
Mean value of 'LIMIT_BAL1': 167.78574266666666


In [25]:
# create 'Payratio' variable (payment ratio)
data['Payratio'] = data.apply(lambda x: (x['PAY_AMT1'] + x['PAY_AMT2'] + x['PAY_AMT3'] + x['PAY_AMT4'] + x['PAY_AMT5'] + x['PAY_AMT6']) / x['LIMIT_BAL1'], axis=1)

# divide the variable into five equal bins
data['Payratio_bins'] = pd.qcut(data['Payratio'], q=5)

# compute default rate in each bin
default_rate = data.groupby(['Payratio_bins'])['default payment next month'].mean()

# print the results
print(default_rate)

Payratio_bins
(0.001, 49.519]         0.246333
(49.519, 121.911]       0.195667
(121.911, 190.182]      0.232667
(190.182, 307.396]      0.248333
(307.396, 14530.112]    0.183000
Name: default payment next month, dtype: float64


In [26]:
# create rescaled variable
data['LIMIT_BAL1'] = data['Limit'] / 1000

# divide the variable into five equal bins
data['Limit_bins'] = pd.qcut(data['LIMIT_BAL1'], q=5)

# compute default rate in each bin
default_rate = data.groupby(['Limit_bins'])['default payment next month'].mean()

# print the results
print(default_rate)

Limit_bins
(10.099, 50.3]     0.327515
(50.3, 100.3]      0.261679
(100.3, 180.2]     0.205867
(180.2, 270.5]     0.168276
(270.5, 1000.5]    0.137966
Name: default payment next month, dtype: float64


In [27]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Acc_ID                      30000 non-null  int64   
 1   Limit                       30000 non-null  int64   
 2   SEX                         30000 non-null  int64   
 3   EDUCATION                   30000 non-null  int64   
 4   MARRIAGE                    30000 non-null  int64   
 5   Debt                        29999 non-null  float64 
 6   BILL_AMT1                   30000 non-null  int64   
 7   BILL_AMT2                   30000 non-null  int64   
 8   BILL_AMT3                   30000 non-null  int64   
 9   BILL_AMT4                   30000 non-null  int64   
 10  BILL_AMT5                   30000 non-null  int64   
 11  BILL_AMT6                   30000 non-null  int64   
 12  PAY_AMT1                    30000 non-null  int64   
 13  PAY_AMT2        

In [28]:
# Create a new column 'Debt_bins' by dividing 'Debt' variable into five equal bins
data['Debt_bins'] = pd.cut(data['Debt'], 5)

# Compute default rate in each bin
default_rate = data.groupby('Debt_bins')['default payment next month'].mean()

print(default_rate)

Debt_bins
(21.941, 33.8]    0.219991
(33.8, 45.6]      0.211702
(45.6, 57.4]      0.239583
(57.4, 69.2]      0.269285
(69.2, 81.0]      0.238095
Name: default payment next month, dtype: float64


In [29]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Acc_ID                      30000 non-null  int64   
 1   Limit                       30000 non-null  int64   
 2   SEX                         30000 non-null  int64   
 3   EDUCATION                   30000 non-null  int64   
 4   MARRIAGE                    30000 non-null  int64   
 5   Debt                        29999 non-null  float64 
 6   BILL_AMT1                   30000 non-null  int64   
 7   BILL_AMT2                   30000 non-null  int64   
 8   BILL_AMT3                   30000 non-null  int64   
 9   BILL_AMT4                   30000 non-null  int64   
 10  BILL_AMT5                   30000 non-null  int64   
 11  BILL_AMT6                   30000 non-null  int64   
 12  PAY_AMT1                    30000 non-null  int64   
 13  PAY_AMT2        

In [30]:
print("1st and 99th percentile value of Payment ratio: ")
print(data.Payratio.describe(percentiles=[0.01,0.99]))
print("1st and 99th percentile value of Credit limit: ")
print(data.LIMIT_BAL1.describe(percentiles=[0.01,0.99]))
print("1st and 99th percentile value of Debt: ")
print(data.Debt.describe(percentiles=[0.01,0.99]))

1st and 99th percentile value of Payment ratio: 
count    30000.000000
mean       232.126387
std        313.170522
min          0.001998
1%           0.019933
50%        156.110630
99%       1406.386502
max      14530.112219
Name: Payratio, dtype: float64
1st and 99th percentile value of Credit limit: 
count    30000.000000
mean       167.785743
std        129.746714
min         10.100000
1%          10.400000
50%        140.400000
99%        500.500000
max       1000.500000
Name: LIMIT_BAL1, dtype: float64
1st and 99th percentile value of Debt: 
count    29999.000000
mean        36.985533
std          9.235147
min         22.000000
1%          23.000000
50%         35.000000
99%         62.000000
max         81.000000
Name: Debt, dtype: float64


In [31]:
# Calculate 99th percentile values
limit_99 = np.percentile(data['Limit'], 99)
payratio_99 = np.percentile(data['Payratio'], 99)
debt_99 = np.percentile(data['Debt'], 99)

# Create capped variables
data['Limit_cap'] = np.where(data['Limit'] > limit_99, limit_99, data['Limit'])
data['Payratio_cap'] = np.where(data['Payratio'] > payratio_99, payratio_99, data['Payratio'])
data['Debt_cap'] = np.where(data['Debt'] > debt_99, debt_99, data['Debt'])

# Rescale credit limit
data['Limit_rescaled'] = data['Limit_cap'] / 1000

# Calculate mean before and after capping
pre_mean = data[['Payratio', 'Limit', 'Debt']].mean()
post_mean = data[['Payratio_cap', 'Limit_cap', 'Debt_cap']].mean()

print("Pre-capped mean:\n", pre_mean)
print("\nPost-capped mean:\n", post_mean)

Pre-capped mean:
 Payratio       232.126387
Limit       167785.742667
Debt            36.985533
dtype: float64

Post-capped mean:
 Payratio_cap       225.715882
Limit_cap       167111.469333
Debt_cap            36.985533
dtype: float64


In [32]:
data['intercept'] = 1
intercept_mean = data.intercept.mean()
print("Mean of Intercept variable: ", intercept_mean)

Mean of Intercept variable:  1.0


In [33]:
from sklearn.model_selection import train_test_split
random_seed = 0

In [34]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Acc_ID                      30000 non-null  int64   
 1   Limit                       30000 non-null  int64   
 2   SEX                         30000 non-null  int64   
 3   EDUCATION                   30000 non-null  int64   
 4   MARRIAGE                    30000 non-null  int64   
 5   Debt                        29999 non-null  float64 
 6   BILL_AMT1                   30000 non-null  int64   
 7   BILL_AMT2                   30000 non-null  int64   
 8   BILL_AMT3                   30000 non-null  int64   
 9   BILL_AMT4                   30000 non-null  int64   
 10  BILL_AMT5                   30000 non-null  int64   
 11  BILL_AMT6                   30000 non-null  int64   
 12  PAY_AMT1                    30000 non-null  int64   
 13  PAY_AMT2        

In [35]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split

# Creating X and y variables
X = data[['Debt_cap', 'Limit_cap', 'intercept', 'Payratio_cap']]
y = data['default payment next month']

# Creating new train and test data with random seed as 0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Printing the number of records in train and test data
print("Number of records in train data:", len(X_train))
print("Number of records in test data:", len(X_test))


Number of records in train data: 24000
Number of records in test data: 6000


In [36]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Acc_ID                      30000 non-null  int64   
 1   Limit                       30000 non-null  int64   
 2   SEX                         30000 non-null  int64   
 3   EDUCATION                   30000 non-null  int64   
 4   MARRIAGE                    30000 non-null  int64   
 5   Debt                        29999 non-null  float64 
 6   BILL_AMT1                   30000 non-null  int64   
 7   BILL_AMT2                   30000 non-null  int64   
 8   BILL_AMT3                   30000 non-null  int64   
 9   BILL_AMT4                   30000 non-null  int64   
 10  BILL_AMT5                   30000 non-null  int64   
 11  BILL_AMT6                   30000 non-null  int64   
 12  PAY_AMT1                    30000 non-null  int64   
 13  PAY_AMT2        

In [37]:
print(np.any(np.isnan(X_train)))
print(np.any(np.isinf(X_train)))

True
False


In [38]:
import statsmodels.api as sm

# Check for missing and infinite values in X_train
print(np.any(np.isnan(X_train)))
print(np.any(np.isinf(X_train)))

# Impute missing values
X_train = pd.DataFrame(X_train).fillna(X_train.mean()).values

# Replace infinite values with a large finite value (e.g. 1e9)
X_train[np.isinf(X_train)] = 1e9

# Fit logistic model
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

# Print model coefficients
print(result.summary())

True
False
Optimization terminated successfully.
         Current function value: 0.510306
         Iterations 6
                               Logit Regression Results                               
Dep. Variable:     default payment next month   No. Observations:                24000
Model:                                  Logit   Df Residuals:                    23996
Method:                                   MLE   Df Model:                            3
Date:                        Mon, 13 Mar 2023   Pseudo R-squ.:                 0.03199
Time:                                08:19:32   Log-Likelihood:                -12247.
converged:                               True   LL-Null:                       -12652.
Covariance Type:                    nonrobust   LLR p-value:                3.710e-175
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0087      0.002 

In [39]:
import numpy as np

# Check for missing and infinite values
print(np.isnan(X_train).sum())
print(np.isinf(X_train).sum())

# Replace missing values with the median
X_train[np.isnan(X_train)] = np.nanmedian(X_train)

# Replace infinite values with a large value
X_train[np.isinf(X_train)] = 1e8

# Fit logistic model
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

# Print model coefficients
print(result.summary())


0
0
Optimization terminated successfully.
         Current function value: 0.510306
         Iterations 6
                               Logit Regression Results                               
Dep. Variable:     default payment next month   No. Observations:                24000
Model:                                  Logit   Df Residuals:                    23996
Method:                                   MLE   Df Model:                            3
Date:                        Mon, 13 Mar 2023   Pseudo R-squ.:                 0.03199
Time:                                08:19:32   Log-Likelihood:                -12247.
converged:                               True   LL-Null:                       -12652.
Covariance Type:                    nonrobust   LLR p-value:                3.710e-175
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0087      0.002      5.

In [40]:
import statsmodels.api as sm

# Fit logistic model
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

# Print model coefficients
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.510306
         Iterations 6
                               Logit Regression Results                               
Dep. Variable:     default payment next month   No. Observations:                24000
Model:                                  Logit   Df Residuals:                    23996
Method:                                   MLE   Df Model:                            3
Date:                        Mon, 13 Mar 2023   Pseudo R-squ.:                 0.03199
Time:                                08:19:33   Log-Likelihood:                -12247.
converged:                               True   LL-Null:                       -12652.
Covariance Type:                    nonrobust   LLR p-value:                3.710e-175
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0087      0.002      5.207 

In [41]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Acc_ID                      30000 non-null  int64   
 1   Limit                       30000 non-null  int64   
 2   SEX                         30000 non-null  int64   
 3   EDUCATION                   30000 non-null  int64   
 4   MARRIAGE                    30000 non-null  int64   
 5   Debt                        29999 non-null  float64 
 6   BILL_AMT1                   30000 non-null  int64   
 7   BILL_AMT2                   30000 non-null  int64   
 8   BILL_AMT3                   30000 non-null  int64   
 9   BILL_AMT4                   30000 non-null  int64   
 10  BILL_AMT5                   30000 non-null  int64   
 11  BILL_AMT6                   30000 non-null  int64   
 12  PAY_AMT1                    30000 non-null  int64   
 13  PAY_AMT2        

In [42]:
print(np.any(np.isnan(X_train)))
print(np.any(np.isinf(X_train)))

False
False


In [43]:
# Importing necessary libraries
import statsmodels.api as sm

# Fitting the logistic model using the train data
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

# Printing the model coefficients
print(result.params)

Optimization terminated successfully.
         Current function value: 0.510306
         Iterations 6
x1       0.008734
x2      -0.000004
const   -0.812944
x3      -0.000846
dtype: float64


In [44]:
# Importing necessary libraries
from sklearn.linear_model import LogisticRegression

# Fitting the logistic model using the new train data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Printing the model coefficients
print(logreg.coef_)

[[-1.08369378e-04 -5.56620467e-06 -3.33316382e-06 -1.58954005e-03]]


In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Split data into X and y
# Fit logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Predict probabilities and calculate AUC for train and test data
y_train_prob = logreg.predict_proba(X_train)[:, 1]
y_test_prob = logreg.predict_proba(X_test)[:, 1]
train_auc = roc_auc_score(y_train, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print AUC values for train and test data
print('Train AUC:', train_auc)
print('Test AUC:', test_auc)


Train AUC: 0.6301117469102212
Test AUC: 0.6291968585525504


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Fit logistic regression model
logreg_new = LogisticRegression()
logreg_new.fit(X_train, y_train)

# Predict probabilities and calculate AUC for new train and new test data
y_new_train_prob = logreg_new.predict_proba(X_train)[:, 1]
y_new_test_prob = logreg_new.predict_proba(X_test)[:, 1]
new_train_auc = roc_auc_score(y_train, y_new_train_prob)
new_test_auc = roc_auc_score(y_test, y_new_test_prob)

# Print AUC values for new train and new test data
print('New Train AUC:', new_train_auc)
print('New Test AUC:', new_test_auc)


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


New Train AUC: 0.6301117469102212
New Test AUC: 0.6291968585525504


In [47]:
print(np.any(np.isnan(y_train)))
print(np.any(np.isinf(y_train)))

False
False


In [49]:
# importing necessary libraries
from sklearn.metrics import accuracy_score, roc_auc_score

# predicting probabilities on train data
train_prob = logreg .predict_proba(X_train)[:, 1]

# calculating the minimum probability
min_prob = min(train_prob)
print("Minimum probability:", min_prob)

# calculating the maximum probability
max_prob = max(train_prob)
print("Maximum probability:", max_prob)

# converting probabilities to binary predictions using 0.5 threshold
train_pred = (train_prob >= 0.5).astype(int)

# calculating total predicted defaults
total_defaults = sum(train_pred)
print("Total predicted defaults:", total_defaults)



Minimum probability: 0.006520409127041902
Maximum probability: 0.48519199960345116
Total predicted defaults: 0


In [50]:
# predicting probabilities on new train data
new_train_prob = logreg.predict_proba(X_train)[:, 1]

# calculating the minimum probability
min_prob = min(new_train_prob)
print("Minimum probability:", min_prob)

# calculating the maximum probability
max_prob = max(new_train_prob)
print("Maximum probability:", max_prob)

# converting probabilities to binary predictions using 0.5 threshold
new_train_pred = (new_train_prob >= 0.5).astype(int)

# calculating total predicted defaults
total_defaults = sum(new_train_pred)
print("Total predicted defaults:", total_defaults)

# calculating the predicted default rate
default_rate = total_defaults / len(new_train_pred)
print("Predicted default rate:", default_rate)


Minimum probability: 0.006520409127041902
Maximum probability: 0.48519199960345116
Total predicted defaults: 0
Predicted default rate: 0.0
